# **Neural Style Transfer**



### **Importing Modules**

In [0]:
import keras.backend as K
from keras.applications import VGG19
import numpy as np
import time
import scipy.misc
from scipy.optimize import fmin_l_bfgs_b


### **Content Loss**

In [0]:
def content_loss(target_content, gen_image):
  return 0.5 * K.sum(K.square(target_content - gen_image))

### **Gram Matrix**

In [0]:
def gram_matrix(features):
  return K.dot(features, K.transpose(features))

### **Style Loss**

In [0]:
def style_loss(target_style, gen_image):
    style_features = K.batch_flatten(K.permute_dimensions(target_style, (2, 0, 1)))
    generated_features = K.batch_flatten(K.permute_dimensions(gen_image, (2, 0, 1)))
    style_gram_matrix = gram_matrix(style_features)
    generated_gram_matrix = gram_matrix(generated_features)
    return K.sum(K.square(style_gram_matrix - generated_gram_matrix)) / (4.0 * (image_size ** 2) * (image_channels ** 2))

### **Variational Loss**

In [0]:
def variational_loss(gen_image):
    a = K.square(gen_image[:, :image_height-1, :image_width-1, :] - gen_image[:, :image_height-1, 1:, :])
    b = K.square(gen_image[:, :image_height-1, :image_width-1, :] - gen_image[:, 1:, :image_width-1, :])
    return K.sum(K.pow(a + b, 1.25))

### **Total Loss**

In [0]:
def total_loss(gen_image, content_layer, style_layers,content_weight, style_weight, variation_weight):

    content_features = content_layer[0, :, :, :]
    generated_features = content_layer[2, :, :, :]
    c_loss = content_loss(content_features, generated_features)

    s_loss = None
    for layer in style_layers:
        style_features = layer[1, :, :, :]
        generated_features = layer[2, :, :, :]
        if s_loss is None:
            s_loss = style_loss(style_features, generated_features) * (style_weight / len(style_layers))
        else:
            s_loss += style_loss(style_features, generated_features) * (style_weight / len(style_layers))
    v_loss = variational_loss(gen_image)
    return content_weight * c_loss + s_loss * style_weight + variation_weight * v_loss
  


### **Processing Image**

In [0]:
def process_image(image):
    image = image.resize((image_width, image_height))
    image = np.asarray(image, dtype='float32')
    image = np.expand_dims(image, axis=0)
    image = image[:, :, :, :3]
    image[:, :, :, 0] -= 103.939
    image[:, :, :, 1] -= 116.779
    image[:, :, :, 2] -= 123.68
    image = image[:, :, :, ::-1]
    return image

### **Save Image**

In [0]:
from matplotlib import pyplot
def save_image(filename, generated):
    generated = generated.reshape((image_height, image_width, 3))
    generated = generated[:, :, ::-1]
    generated[:, :, 0] += 103.939
    generated[:, :, 1] += 116.779
    generated[:, :, 2] += 123.68
    generated = np.clip(generated, 0, 255).astype('uint8')
    pyplot.imshow(generated)
    pyplot.savefig(filename)
    pyplot.close()

In [0]:
from PIL import Image
style_weight = 1.0
content_weight = 0.025
variation_weight = 1.0
image_height = 512
image_width = 512
image_channels = 3
image_size = image_width * image_height
content_image = Image.open("/content/content1.png")
content_image = process_image(content_image)
style_image = Image.open("/content/style1.jpg")
style_image = process_image(style_image)
gen_image = np.random.uniform(0, 255, (1, image_height, image_width, 3)) - 128
style = K.variable(style_image)
content = K.variable(content_image)
gen_image = K.placeholder((1, image_height, image_width, 3))
loss = K.variable(0.)
input_tensor = K.concatenate([content, style, gen_image], axis=0)
model = VGG19(input_tensor=input_tensor, weights='imagenet', include_top=False)
layers = dict([(layer.name, layer.output) for layer in model.layers])
content_layer = layers['block2_conv2']
style_layers = ['block1_conv2', 'block2_conv2', 'block3_conv3', 'block4_conv3', 'block5_conv3']
style_layers = [layers[layer] for layer in style_layers]
loss = total_loss(gen_image,content_layer, style_layers, content_weight, style_weight,variation_weight)
grads = K.gradients(loss, gen_image)
outputs = [loss]
outputs += grads
f_outputs = K.function([gen_image], outputs)

### **Evaluator Class**

In [0]:
def eval_loss_and_grads(generated):
    generated = generated.reshape((1, image_height, image_width, 3))
    outs = f_outputs([generated])
    loss_value = outs[0]
    grad_values = outs[1].flatten().astype('float64')
    return loss_value, grad_values

class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grad_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values


### **Optimization**

In [0]:
evaluator = Evaluator()
iterations = 50
generated_image = np.random.uniform(0, 255, (1, image_height, image_width, 3)) - 128
for i in range(iterations):
    print('Iteration:', i)
    start_time = time.time()
    generated_image, min_val, info = fmin_l_bfgs_b(evaluator.loss, generated_image.flatten(),
                                                 fprime=evaluator.grads, maxfun=20)
    print('Loss:', min_val)
    end_time = time.time()
    print('Iteration {} took {} seconds'.format(i, end_time - start_time))
    name = "/content/generated/generated_image%04d" % (i+1)
    if((i+1)%10==0):
      save_image(name, generated_image)
      print('Saved image to: {}'.format(name))

Iteration: 0
Loss: 13193906000.0
Iteration 0 took 17.213642358779907 seconds
Iteration: 1
Loss: 9242053000.0
Iteration 1 took 16.348997592926025 seconds
Iteration: 2
Loss: 7180179000.0
Iteration 2 took 16.56907320022583 seconds
Iteration: 3
Loss: 6269006000.0
Iteration 3 took 16.631929397583008 seconds
Iteration: 4
Loss: 5717539000.0
Iteration 4 took 16.74712562561035 seconds
Iteration: 5
Loss: 5392972300.0
Iteration 5 took 16.77759552001953 seconds
Iteration: 6
Loss: 5177689000.0
Iteration 6 took 16.78388237953186 seconds
Iteration: 7
Loss: 5050682400.0
Iteration 7 took 16.686538696289062 seconds
Iteration: 8
Loss: 4956804600.0
Iteration 8 took 16.761302709579468 seconds
Iteration: 9
Loss: 4874658000.0
Iteration 9 took 16.87691903114319 seconds
Saved image to: /content/generated/generated_image0010
Iteration: 10
Loss: 10021169000.0
Iteration 10 took 16.54092288017273 seconds
Iteration: 11
Loss: 7116305400.0
Iteration 11 took 16.714433431625366 seconds
Iteration: 12
Loss: 6129674000.0


In [0]:
!zip -r /content/file3.zip /content/generated

  adding: content/generated/ (stored 0%)
  adding: content/generated/generated_image0160.png (deflated 0%)
  adding: content/generated/generated_image0130.png (deflated 0%)
  adding: content/generated/generated_image0180.png (deflated 0%)
  adding: content/generated/generated_image0040.png (deflated 0%)
  adding: content/generated/generated_image0170.png (deflated 0%)
  adding: content/generated/generated_image0020.png (deflated 0%)
  adding: content/generated/generated_image0010.png (deflated 0%)
  adding: content/generated/generated_image0050.png (deflated 0%)
  adding: content/generated/generated_image0080.png (deflated 0%)
  adding: content/generated/generated_image0100.png (deflated 0%)
  adding: content/generated/generated_image0120.png (deflated 0%)
  adding: content/generated/generated_image0090.png (deflated 0%)
  adding: content/generated/generated_image0030.png (deflated 0%)
  adding: content/generated/generated_image0150.png (deflated 0%)
  adding: content/generated/generat

In [0]:
from google.colab import files
files.download("/content/file3.zip")